In [89]:
import mesa
from functional import seq
import random

from dataclasses import dataclass

In [90]:
from typing import Tuple, Optional, List

In [91]:
@dataclass
class Hotel():
    id: int
    cost: float
    max_capacity: int
    occupied: int = 0

    # stats
    new_rev: int = 0
    total_rev: int = 0

    def remaining_capacity(self) -> int:
        return self.max_capacity - self.occupied

In [92]:
@dataclass
class PointOfInterest():
    id: int
    cost: float
    max_capacity: int
    occupied: int = 0

    # stats
    visit: int = 0
    total_visit: int = 0

    def remaining_capacity(self) -> int:
        return self.max_capacity - self.occupied

In [93]:
class Tourist(mesa.Agent):
    def __init__(self, unique_id: int, model, loc: int, trip_length: int):
        super().__init__(unique_id, model)
        self.loc: int = loc
        self.hotel: Optional[Hotel] = None
        self.trip_length: int = trip_length
        self.remaining_life: int = trip_length

        self.satisfaction: int = 0

    def step(self):
        self.remaining_life -=1
        if self.hotel is None and self.remaining_life >= 3:
            print("Looking for hotel")
            hotels = (
                seq(self.model.hotels)
                    .filter(lambda h: h.remaining_capacity() >= 0)
                    .list()
            )

            if len(hotels) <= 0:
                self.remaining_life = -1
                return;

            random.shuffle(hotels)

            self.hotel = hotels[0]
            self.hotel.occupied += 1
            self.hotel.new_rev += 1
            self.hotel.total_rev += 1

            return

        # go to a POINT OF INTEREST


In [ ]:
@dataclass
class TouristInflow():
    id: int
    cost: float
    max_capacity: int
    occupied: int = 0

    # stats
    new_rev: int = 0
    total_rev: int = 0

    def remaining_capacity(self) -> int:
        return self.max_capacity - self.occupied

In [94]:
class Model(mesa.Model):
    def __init__(self, num_tourists, num_hotels, num_of_poi):
        self.hotels: List[Hotel] = []
        self.pois: List[PointOfInterest] = []
        self.tourists: List[Tourist] = []

        self.schedule=mesa.time.RandomActivation(self)

        # Stats
        
        
        # should be replaced with data from file
        for i in range (num_of_poi):
            self.hotels.append(Hotel(i, 200, 100))

        for i in range (num_hotels):
            self.pois.append(PointOfInterest(i, 200, 100))
            # self.schedule.add(hotel)

        for i in range(num_tourists):
            tourist = Tourist(i, self, 0, 10)
            self.tourists.append(tourist)
            self.schedule.add(tourist)

    def step(self):
        # self.schedule.step()
        # self.schedule.step()
        # self.schedule.step()

        self.update_tourists()
        self.update_tourists()
        self.update_tourists()

        self.remove_tourists()

        print("stats")
        print(self.hotel_stats())

    def update_tourists(self):
        random.shuffle(self.tourists)

        for tourist in self.tourists:
            tourist.step()



    def remove_tourists(self):
        # (
        #     seq(self.model.schedule.agents)
        #         .filter()
        # )
        remove_index: list[int] = (seq(self.tourists)
            .enumerate()
            .filter(lambda x: x[1].remaining_life < 0)
            .map(lambda x: x[0])
            .sorted(reverse=True)
            .list()
        )
        for i1 in remove_index:
            self.tourists[i1].hotel.occupied -=1

            del self.tourists[i1]

    def hotel_stats(self) -> Tuple[float, float, float]:
        total_capacity: int = (seq(self.hotels)
            .map(lambda h: h.max_capacity)
            .fold_left(0, lambda acc, next: acc + next)
        )
        empty_rooms: int = (seq(self.hotels)
            .map(lambda h: h.remaining_capacity())
            .fold_left(0, lambda acc, next: acc + next)
        )
        occupied_rooms: int = (seq(self.hotels)
            .map(lambda h: h.occupied)
            .fold_left(0, lambda acc, next: acc + next)
        )
        
        new_reservations: int = (seq(self.hotels)
            .map(lambda h: h.new_rev)
            .fold_left(0, lambda acc, next: acc + next)
        )
        for hotel in self.hotels:
            hotel.new_rev = 0

        return (total_capacity, empty_rooms, occupied_rooms, new_reservations)

In [95]:
starter_model=Model(3, 3, 3)
for i in range(20):
    starter_model.step()

Looking for hotel
Looking for hotel
Looking for hotel
stats
(300, 297, 3, 3)
stats
(300, 297, 3, 0)
stats
(300, 297, 3, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)
stats
(300, 300, 0, 0)


c:\Users\hasin\AppData\Local\Programs\Python\Python311\Lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)
